In [ ]:
#importing of necessary files

import json
current_df = pd.read_json("new.json")
previous_df = pd.read_json("old.json")
import pandas as pd
import nest_asyncio
nest_asyncio.apply()
import discord
from discord.ext import commands

#bot token from discord

TOKEN = "MTAwMTA5ODgwODQ3NTc4MzI4OQ.GzVJxB.7vG_LHenQYY7huddQHtYO4IZKhaisrmNM9bCr0"

#setting up the client

client = commands.Bot(command_prefix="!")

#clearing the 'help' command to allow for own command

client.remove_command("help")

#instance to check if code loaded and is running 

@client.event
async def on_ready():
    print('We have logged in as {0.user}'.format(client))

#greeting message sent to users
#creating bot commands
##!help command - gives a brief description of the discord bot and the bot commands that may be used

@client.command()
async def help(ctx):
    string = """
    Welcome to currentsee, where you can not only see the current currency exchange rates but compare it with the recent!
    
    Here are the commands you can use to fully utilize currentsee:
    !help - brief description of currentsee + bot commands
    !currencies - list of currencies + their respective codes
    !rates - quick exchange rates based on your desired currencies
        (example: to check for the exchange rate of Philippine Pesos to US Dollars type '!rates PHP USD')
    !exchange - quick exchange rate conversions based on your desired amount and currencies
        (example: to convert 50 Philippine Pesos to US Dollars type '!set 50 PHP USD')
    """
    await ctx.send(string)
    return

##!currencies command - shows the list of currencies and their respective codes found in the database

@client.command()
async def currencies(ctx):
    string_name = ' '.join(list(current_df['name']))
    string_code = ' '.join(list(current_df['alphaCode']))
    await ctx.send(string_name)
    await ctx.send(string_code)
    return

#!rates command - where the user inputs the desired currencies they want to check the exchange rate for 

@client.command()
#users will be inputting the original currency and desired converted currency respectively
async def rates(ctx, convert_from, convert_to):
    convert_from.lower()
    convert_to.lower()
    
    #checking if the currency inputted for the original and desired are part of the list
    if convert_from in list(current_df.keys()) and convert_to in list(current_df.keys()):
        
        #when desired currency is USD
        if convert_to == "usd":
            current_exchange_rate = int(current_df[convert_from][rate])
            previous_exchange_rate = int(previous_df[convert_from][rate])
            difference = int(current_exchange_rate) - int(previous_exchange_rate)
        
        #when desired currency is not USD (ex: PHP to USD to CAD)
        else: 
            current_exchange_rate = int(current_df[convert_from][rate]) * int(current_df[convert_to][inverseRate])
            previous_exchange_rate = int(previous_df[convert_from][rate]) * int(previous_df[convert_to][inverseRate])
            difference = int(current_exchange_rate) - int(previous_exchange_rate)
        
        #when the difference is greater than 0, recommendation to convert back to original currency is given 
        if difference > 0:
            difference = abs(difference)
            response = f"""
            The exchange rate from {convert_from} to {convert_to} is currently {current_exchange_rate} and was previously {previous_exchange_rate}.

            Looks like the exchange rate went up by {difference}! It's a good time to convert back to {convert_from} from {convert_to}.
            """
            return response
        
        #when the difference is less than 0, recommendation to convert to desired currency to capitalize is given
        elif difference < 0:
            difference = abs(difference)
            response = f"""
            The exchange rate from {convert_from} to {convert_to} is currently {current_exchange_rate} and was previously {previous_exchange_rate}.

            Looks like the exchange rate went down by {difference}! If you want to capitalize on exchange rates, it's a good time to convert your {convert_from} to {convert_to} and convert back in the future. 
            """
            return response
        
        #when computed difference is 0, recommendation to use the bot again to check for changes is given
        else:
            difference = abs(difference)
            response = f"""
            The exchange rate from {convert_from} to {convert_to} was previously and is currently {previous_exchange_rate}.

            Looks like the exchange rate didn't change from before! Come back tomorrow to check for any changes in the different exchange rates. 
            """
            return response
                                                                                              
    #when currency is not found in the database
    else:
        response = "Oh no! Please make sure that the currencies you wish to input are typed correctly and exist in our database by using the !currencies command."
        return response
    
##!exchange command - where the user inputs their desired amount to be converted from and to a certain currency 
    
@client.command()
#users will be inputting the amount, original currency, and desired converted currency respectively
async def exchange(ctx, amount, convert_from, convert_to):
    convert_from.lower()
    convert_to.lower()
    
    #checking if the currency inputted for the original and desired are part of the list
    if convert_from in list(current_df.keys()) and convert_to in list(current_df.keys()):
        
        #when desired currency is USD, amount is multiplied to the given rate
        if convert_to == "usd":
            current_converted_amount = (amount * int(current_df[convert_from][rate]))
            previous_converted_amount = (amount * int(previous_df[convert_from][rate]))
            difference = int(current_converted_amount - previous_converted_amount)
        
        #when desired currency is not USD, amount is multiplied to the given rate then multiplied to the inverse rate after
            #two conversions occur (ex: PHP to USD to CAD)
        else: 
            current_converted_amount = (amount * int(current_df[convert_from][rate])) * int(current_df[convert_to][inverseRate]
            previous_converted_amount = (amount * int(previous_df[convert_from][rate])) * int(previous_df[convert_to][inverseRate]
            difference = int(current_converted_amount - previous_converted_amount)
        
        #when computed difference is greater than 0, recommendation to convert back to original currency is given                                                                                      
        if difference > 0:
            difference = abs(difference)
            response = f"""
            {amount} {convert_from} is currently {current_converted_amount} {convert_to} and was previously {previous_converted_amount} {convert_to}.
                
            Looks like the conversion went up by {difference}! It's a good time to convert back to {convert_from} from {convert_to}. 
            """
            return response

        #when computed difference is less than 0, recommendation to convert to desired currency to capitalize is given
        elif difference < 0:
            difference = abs(difference)
            response = f"""
            {amount} {convert_from} is currently {current_converted_amount} {convert_to} and was previously {previous_converted_amount} {convert_to}.
                
            Looks like the conversion went down by {difference}! If you want to capitalize on exchange rates, it's a good time to convert your {amount} {convert_from} to {convert_to} and convert back in the future. 
            """
            return response
                                                                                        
        #when computed difference is 0, recommendation to use the bot again to check for changes is given
        else:
            response = f"""
            {amount} {convert_from} was previously and is currently {current_converted_amount} {convert_to}.
                
            Looks like the conversion didn't change from before! Come back tomorrow to check for any changes in the different exchange rates. 
            """
            return response
                                                                                              
    #when currency is not found in the database
    else:
        response = "Oh no! Please make sure that the currencies you wish to input are typed correctly and exist in our database by using the !currencies command."
        return response
                                                                                            
#running the bot through the client   
                                                                                              
client.run(TOKEN)